In [442]:
import pandas as pd
import os
import psycopg2
import re
import sys
import json


In [443]:
# init path
root_path = os.getcwd()
source_path = root_path + "\\data\\source"

In [444]:
# get list of file in source path 
source_files = os.listdir(source_path)

In [445]:
def loop_files(source):
    for file in source:
        
        hdf_path = source_path + "\\" + file
        df = pd.read_hdf(hdf_path)
        etl_file(file, df)

In [446]:
def init_connection():
    try:
        connection = psycopg2.connect(user="postgres", password="s,^ditmt", database="postgres")
        cursor = connection.cursor()
        return cursor, connection
    
    except(Exception, psycopg2.Error) as error:
        print("Failed to connect database", error)

In [564]:
# https://stackoverflow.com/questions/4069718/postgres-insert-if-does-not-exist-already/13342031#13342031
def etl_file(file, df):
    
    # init connection
    cursor, connection = init_connection()
    
    # insert field by each file name
    insert_field_id = insert_career_fields(cursor, connection, file)
    
    # loop thought row
    for index, row in df.iterrows():
        insert_group_id = insert_career_groups(cursor, connection, row.group, insert_field_id)
        insert_position_id = insert_career_positions(cursor, connection, row.position, insert_group_id)
        insert_profile_id = insert_profiles(cursor, connection, row, insert_position_id)
        
        if insert_profile_id:
            insert_skills(cursor, connection, row, insert_position_id)

In [565]:
def insert_skills(cursor, connection, data, insert_position_id):
    
    # if position is already exist skip
    skill_hard = [re.sub(r'[^A-Za-z0-9 ]+', '', skill) for skill in data.skill_hard if len(data.skill_hard)]
    skill_soft = [re.sub(r'[^A-Za-z0-9 ]+', '', skill) for skill in data.skill_soft if len(data.skill_soft)]
    
    insert_skill(cursor, connection, skill_hard, 'Hard Skill')
    insert_skill(cursor, connection, skill_soft, 'Soft Skill')
    insert_position_m_skill(cursor, connection, skill_hard, insert_position_id)
    insert_position_m_skill(cursor, connection, skill_soft, insert_position_id)

In [566]:
def insert_skill(cursor, connection, data, skill_type):
    insert_skill_id = []
    for skill in data:
        skill = skill
        query = f"\
                INSERT INTO skills (name, type) SELECT '{skill}', '{skill_type}' \
                WHERE NOT EXISTS (SELECT name FROM skills WHERE name='{skill}') \
                RETURNING id \
                "
        try:
            cursor.execute(query)
            connection.commit()

            # check inserted
            if cursor.rowcount:
                insert_skill_id.append(cursor.fetchone()[0])
            else:
                query = f"SELECT id FROM skills WHERE name='{skill}'"
                cursor.execute(query)
                connection.commit()
                insert_skill_id.append(cursor.fetchone()[0])

        except (Exception, psycopg2.Error) as error:
            print("Failed to insert `skill`", error)
            
    return insert_skill_id

In [567]:
# ทำอย่างงี้ไม่ได้ ต้องเอา skill ไปหา id ที่ไม่ซ้ำกันก่อน
def insert_position_m_skill(cursor, connection, data, position_id):
    if (len(data)): # validate data is exist
        for skill in data:
            
            # get skill_id
            query = f"SELECT id FROM skills WHERE name='{skill}'"
            cursor.execute(query)
            connection.commit()
            skill_id = cursor.fetchone()[0]
            
            
            query = f"SELECT id FROM position_m_skill WHERE position_id='{position_id}' AND skill_id='{skill_id}'"
            cursor.execute(query)
            connection.commit()
            
            # if duplicate plus amount
            if cursor.rowcount:
                query = f"UPDATE position_m_skill SET amount=amount+1 WHERE position_id='{position_id}' AND skill_id='{skill_id}'"
            else:
                query = f"INSERT INTO position_m_skill (position_id, skill_id) VALUES ({position_id}, {skill_id})"
                
            cursor.execute(query)
            connection.commit()
        return True
    else:
        return False

In [568]:
def insert_career_positions(cursor, connection, data, field_id):
    query = f"\
        INSERT INTO career_positions (name, career_group_id) SELECT '{data}', {field_id} \
        WHERE NOT EXISTS (SELECT name FROM career_positions WHERE name='{data}') \
        RETURNING id \
        "
    try:
        cursor.execute(query)
        connection.commit()
        
        # check inserted
        if cursor.rowcount:
            return cursor.fetchone()[0]
        else:
            query = f"SELECT id FROM career_positions WHERE name='{data}'"
            cursor.execute(query)
            connection.commit()
            return cursor.fetchone()[0]
        
    except (Exception, psycopg2.Error) as error:
        print("Failed to insert `career_groups`", error)

In [569]:
def insert_career_groups(cursor, connection, data, field_id):
    query = f"\
        INSERT INTO career_groups (name, career_field_id) SELECT '{data}', {field_id} \
        WHERE NOT EXISTS (SELECT name FROM career_groups WHERE name='{data}') \
        RETURNING id \
        "
    try:
        cursor.execute(query)
        connection.commit()
        
        # check inserted
        if cursor.rowcount:
            return cursor.fetchone()[0]
        else:
            query = f"SELECT id FROM career_groups WHERE name='{data}'"
            cursor.execute(query)
            connection.commit()
            return cursor.fetchone()[0]
        
    except (Exception, psycopg2.Error) as error:
        print("Failed to insert `career_groups`", error)

In [570]:
def insert_career_fields(cursor, connection, data):
    
    val = data.replace(".h5", "")
    query = f"\
        INSERT INTO career_fields (name) SELECT ('{val}') \
        WHERE NOT EXISTS (SELECT id FROM career_fields WHERE name = '{val}') \
        RETURNING id \
        "
    try:
        cursor.execute(query)
        connection.commit()
        
        # check inserted
        if cursor.rowcount:
            return cursor.fetchone()[0]
        else:
            query = f"SELECT id FROM career_fields WHERE name='{val}'"
            cursor.execute(query)
            connection.commit()
            return cursor.fetchone()[0]
        
    except (Exception, psycopg2.Error) as error:
        print("Failed to insert `career_fields`", error)

In [571]:
print("\''")

''


In [572]:
def list_to_postgres_array(data):
    tmp = str(data).replace('[', '{').replace(']', '}').replace('\'', '\"').replace('"', '\'').replace("\''", '\'')
    return tmp

In [579]:

def insert_profiles(cursor, connection, user, position_id):
    if "education" in user:
        education = list_to_postgres_array(user.education)
    else:
        education = []
    
    exp = str(user.exp).replace('\'', '\"').replace('"', '\'').replace("\''", '\'')
    skill_hard = list_to_postgres_array(user.skill_hard)
    skill_soft = list_to_postgres_array(user.skill_soft)
    interest = list_to_postgres_array(user.interest)
    
    query = f"""\
        INSERT INTO profiles (url, about, exp, skill_hard, skill_soft, career_position_id, interest, education) \
        SELECT %s, %s, %s, %s, %s, %s, %s, %s\
        WHERE NOT EXISTS (SELECT id FROM profiles WHERE url = %s)
        RETURNING id \
        """
    try:
        cursor.execute(query, (user.url, user.about, exp, skill_hard, skill_soft, position_id, interest, education, user.url))
        connection.commit()
        if cursor.rowcount:
            return cursor.fetchone()[0]
        else:
            return False
        
    except (Exception, psycopg2.Error) as error:
        print("Failed to insert `profiles` - SKIP", error)
        connection.rollback()
        return False

In [582]:
loop_files(source_files)

Failed to insert `profiles` - SKIP malformed array literal: "{'Root BD  {Intellectual Capital Network}', 'Asian Institute of Technology (AIT) Alumni Business Network', 'USAID', 'Microsoft IoT', 'AquacultureHub', 'Melinda Gates'}"
LINE 1: ...climate change''}', '{''capacity building''}', 5, '{''Root B...
                                                             ^
DETAIL:  Unexpected "{" character.

Failed to insert `profiles` - SKIP malformed array literal: "{'Prince of Songkla University', 'True Corporation', 'Red Hat OpenShift', 'Actuary Network - The #1 Group for Actuaries', 'DevOps | SRE | AIOps | MLOps {Cloud, Kubernetes, Docker, Microservices, Gitops} Discussions', 'AWS Cloud Computing (For Interested Parties & Users)'}"
LINE 1: ...', ''openshift'', ''aws''}', '{''teamwork''}', 8, '{''Prince...
                                                             ^
DETAIL:  Unexpected "{" character.

Failed to insert `profiles` - SKIP malformed array literal: "{'LINE MAN Wongnai', 'SCB 

Failed to insert `profiles` - SKIP malformed array literal: "{'China Expats & Returnees（Business opportunities; employment; and life networking）{OneTube/JobTube}', 'NUS Postgraduate Alumni', 'Savory Institute', 'Facebook', 'IBM', 'National University of Singapore (NUS) Alumni'}"
LINE 1: ...mining'', ''computational biology''}', '{}', 111, '{''China ...
                                                             ^
DETAIL:  Unexpected "{" character.

Failed to insert `profiles` - SKIP malformed array literal: "{'RAIL and TRANSPORT {110,000+ Largest Rail & Transport Group}', 'Winton Associates Pty Ltd', 'Rail Civil Engineering', 'University of Adelaide'}"
LINE 1: ...vil engineering'', ''construction''}', '{}', 113, '{''RAIL a...
                                                             ^
DETAIL:  Unexpected "{" character.

Failed to insert `profiles` - SKIP malformed array literal: "{'Ticketing & Passengers Information Systems(TPIS)', 'The University of Tokyo', 'RAIL and TRANSPORT {11

Failed to insert `profiles` - SKIP malformed array literal: "{'Mahidol University', 'Cisco', 'Huawei', 'Satya Nadella', 'Jeff Weiner', 'KASIKORN Business-Technology Group {KBTG}'}"
LINE 1: ... '{''english as a second language (esl)''}', 149, '{''Mahido...
                                                             ^
DETAIL:  Unexpected "{" character.

Failed to insert `profiles` - SKIP malformed array literal: "{'KASIKORN Business-Technology Group {KBTG}', 'Kasetsart University'}"
LINE 1: ...'kubernetes'', ''istio'', ''linux''}', '{}', 149, '{''KASIKO...
                                                             ^
DETAIL:  Unexpected "{" character.

Failed to insert `profiles` - SKIP malformed array literal: "{'Cisco', 'Satya Nadella', 'Sachin Tendulkar', 'Docker Professionals Network ▶️ Container Infrastructure Architects Developers DevOps Engineers', 'Gary Vaynerchuk', 'DevOps | SRE | AIOps | MLOps {Cloud, Kubernetes, Docker, Microservices, Gitops} Discussions'}"
LINE 1: ..., ''re

Failed to insert `profiles` - SKIP malformed array literal: "{'KASIKORN Business-Technology Group {KBTG}', 'K2', 'Atos', 'Kasetsart University', 'I&I Group PLC. - iiG', 'Deloitte Consulting'}"
LINE 1: ...rts 10.0'', ''k2'', ''sql'', ''c#''}', '{}', 167, '{''KASIKO...
                                                             ^
DETAIL:  Unexpected "{" character.

Failed to insert `profiles` - SKIP malformed array literal: "{'FNZ Group', 'KASIKORN Business-Technology Group {KBTG}', 'SCB – Siam Commercial Bank', 'Stream I.T. Consulting Ltd.', 'IBM', 'King Mongkut's University of Technology Thonburi'}"
LINE 1: ...le methodologies''}', '{''communication''}', 168, '{''FNZ Gr...
                                                             ^
DETAIL:  Unexpected "{" character.

Failed to insert `profiles` - SKIP malformed array literal: "{'Oracle ERP User Network', 'Symantec', 'Oracle Contractors {28,000+}', 'Universidad de San Carlos de Guatemala', 'Software and Cloud - Pricing and Licensing

Failed to insert `profiles` - SKIP malformed array literal: "{'SCB – Siam Commercial Bank', 'Krungsri', 'Aetna, a CVS Health Company', 'KASIKORN Business-Technology Group {KBTG}', 'SCG', 'Coursera Course Certificates'}"
LINE 1: ...cript'', ''c#'', ''.net framework''}', '{}', 209, '{''SCB – ...
                                                             ^
DETAIL:  Unexpected "{" character.

Failed to insert `profiles` - SKIP malformed array literal: "{'SCB – Siam Commercial Bank', 'KASIKORN Business-Technology Group {KBTG}', 'KASIKORNBANK', 'Agoda', 'Sertis', 'King Mongkut's University of Technology Thonburi'}"
LINE 1: ...cript'', ''cisco systems products''}', '{}', 210, '{''SCB – ...
                                                             ^
DETAIL:  Unexpected "{" character.

Failed to insert `profiles` - SKIP malformed array literal: "{'AIS - Advanced Info Services Plc.', 'ANCOR THAILAND', 'KASIKORN Business-Technology Group {KBTG}', 'Tri Petch IT Solutions Co.,Ltd.', 'Kasetsart

Failed to insert `profiles` - SKIP malformed array literal: "{'KASIKORN Business-Technology Group {KBTG}', 'Saitech Solutions', 'Saitech IT Solutions', 'LinkedIn Guide to Networking'}"
LINE 1: ...ile develop , support user''}]', '{}', '{}', 225, '{''KASIKO...
                                                             ^
DETAIL:  Unexpected "{" character.

Failed to insert `profiles` - SKIP malformed array literal: "{'SCB – Siam Commercial Bank', 'IBM', 'SCG', 'KASIKORN Business-Technology Group {KBTG}', 'CIMB Thai Bank'}"
LINE 1: ...acle sql developer'', ''filezilla''}', '{}', 225, '{''SCB – ...
                                                             ^
DETAIL:  Unexpected "{" character.

Failed to insert `profiles` - SKIP malformed array literal: "{'KASIKORN Business-Technology Group {KBTG}', 'Web Designer and UI Developer', 'LTMAN', 'TechCrunch', 'React Developers - ReactJS & React Native Professional Development Mastermind', 'LINE Company (Thailand)'}"
LINE 1: ...vue.js'', ''la

Failed to insert `profiles` - SKIP malformed array literal: "{'India Infrastructure Publishing Private Limited', 'Delhi Metro rail Corporation {DMRC}', 'Aligarh Muslim University', 'Phil Baumann, RN BSN', 'Bob Nardelli', 'QATAR World Cup 2022 Project Opportunities'}"
LINE 1: ...t'', ''project planning'', ''rail''}', '{}', 231, '{''India ...
                                                             ^
DETAIL:  Unexpected "{" character.

Failed to insert `profiles` - SKIP malformed array literal: "{'Future Technology: Artificial Intelligence, Robotics, IoT, Blockchain, Bitcoin | Startups (BIG)', 'Oil & Energy, Construction - HR & Recruitment', 'ASCE: Construction Engineering', 'RAIL and TRANSPORT {110,000+ Largest Rail & Transport Group}', 'BIM (Building Information Modeling) and Architecture, Engineering & Construction PROJECTS and JOBS', 'Research, Methodology, and Statistics in the Social Sciences'}"
LINE 1: ...ing''}', '{''communication'', ''english''}', 231, '{''Future...
        